In [24]:
import pandas as pd
import plotly.express as px
from sklearn.utils import shuffle
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

def extractTimbreMeans(df):
    avgTimbreDf = df[['avg_timbre1','avg_timbre2','avg_timbre3','avg_timbre4',
                      'avg_timbre5','avg_timbre6','avg_timbre7','avg_timbre8',
                      'avg_timbre9','avg_timbre10','avg_timbre11','avg_timbre12']]
    df['avgTimbreMean'] = avgTimbreDf.sum(axis=1)
    varTimbreDf = df[['var_timbre1','var_timbre2','var_timbre3','var_timbre4',
                      'var_timbre5','var_timbre6','var_timbre7','var_timbre8',
                      'var_timbre9','var_timbre10','var_timbre11','var_timbre12']]
    df['varTimbreMean'] = varTimbreDf.sum(axis=1)
    return df

def extractVarTimbre(df):
    varTimbreDf = df[['var_timbre1','var_timbre2','var_timbre3','var_timbre4',
                      'var_timbre5','var_timbre6','var_timbre7','var_timbre8',
                      'var_timbre9','var_timbre10','var_timbre11','var_timbre12']]
    df['mean'] = varTimbreDf.mean(axis=1)
    return df

def sampling(df):
    return df.head(10)

pd.options.mode.chained_assignment = None  # default='warn'

dataset = pd.read_csv('msd_genre_dataset0.csv', header=0)

classicPopAndRock = dataset.iloc[0:23895]
punk = dataset.iloc[23895:27095]
folk = dataset.iloc[27095:40287]
pop = dataset.iloc[40287:41904]
danceAndElectronica = dataset.iloc[41904:46839]
metal = dataset.iloc[46839:48942]
jazz = dataset.iloc[48942:53276]
classical = dataset.iloc[53276:55150]
hipHop = dataset.iloc[55150:55584]
soulAndReggae = dataset.iloc[55584:59600]

classicPopAndRock.name = 'classicPopAndRock'
punk.name = 'punk'
folk.name = 'folk'
pop.name = 'pop'
danceAndElectronica.name = 'danceAndElectronica'
jazz.name = 'jazz'
classical.name = 'classical'
hipHop.name = 'hipHop'
soulAndReggae.name = 'soulAndReggae'

datasetFrames = [sampling(classicPopAndRock),sampling(punk), sampling(folk),sampling(pop),
                 sampling(danceAndElectronica), sampling(metal), sampling(jazz),
                 sampling(classical), sampling(hipHop), sampling(soulAndReggae)]

datasetDataframe = pd.concat(datasetFrames)
datasetSample = shuffle(datasetDataframe)
genreVector = datasetSample[['avg_timbre1','avg_timbre2','avg_timbre3','avg_timbre4',
                             'avg_timbre5','avg_timbre6','avg_timbre7','avg_timbre8',
                             'avg_timbre9','avg_timbre10','avg_timbre11','avg_timbre12',
                             'var_timbre1','var_timbre2','var_timbre3','var_timbre4',
                             'var_timbre5','var_timbre6','var_timbre7','var_timbre8',
                             'var_timbre9','var_timbre10','var_timbre11','var_timbre12', 'genre']]

kMeans = KMeans(n_clusters=10)
kMeans.fit(genreVector[['avg_timbre1','avg_timbre2','avg_timbre3','avg_timbre4',
                             'avg_timbre5','avg_timbre6','avg_timbre7','avg_timbre8',
                             'avg_timbre9','avg_timbre10','avg_timbre11','avg_timbre12',
                             'var_timbre1','var_timbre2','var_timbre3','var_timbre4',
                             'var_timbre5','var_timbre6','var_timbre7','var_timbre8',
                             'var_timbre9','var_timbre10','var_timbre11','var_timbre12']])
testingSet = genreVector
genreVector.loc[:, 'genre'] = kMeans.labels_
results = pd.DataFrame(confusion_matrix(genreVector[['genre']], testingSet[['genre']]),
                       index=['classicPopAndRock', 'punk', 'folk', 'pop', 'danceAndElectronica',
                               'metal', 'jazz', 'classical', 'hipHop', 'soulAndReggae'],
                       columns=['classicPopAndRock', 'punk', 'folk', 'pop', 'danceAndElectronica',
                               'metal', 'jazz', 'classical', 'hipHop', 'soulAndReggae'])
print(results)

                     classicPopAndRock  punk  folk  pop  danceAndElectronica  \
classicPopAndRock                   10     0     0    0                    0   
punk                                 0     3     0    0                    0   
folk                                 0     0     5    0                    0   
pop                                  0     0     0   21                    0   
danceAndElectronica                  0     0     0    0                   12   
metal                                0     0     0    0                    0   
jazz                                 0     0     0    0                    0   
classical                            0     0     0    0                    0   
hipHop                               0     0     0    0                    0   
soulAndReggae                        0     0     0    0                    0   

                     metal  jazz  classical  hipHop  soulAndReggae  
classicPopAndRock        0     0          0       

In [ ]:
# classicPopAndRockAvgTimbre = extractAvgTimbre(classicPopAndRock)
# punkAvgTimbre = extractAvgTimbre(punk)
# folkAvgTimbre = extractAvgTimbre(folk)
# popAvgTimbre = extractAvgTimbre(classicPopAndRock)
# danceAndElectronicaAvgTimbre = extractAvgTimbre(danceAndElectronica)
# metalAvgTimbre = extractAvgTimbre(metal)
# jazzAvgTimbre = extractAvgTimbre(jazz)
# classicalAvgTimbre = extractAvgTimbre(classical)
# hipHopAvgTimbre = extractAvgTimbre(hipHop)
# soulAndReggaeAvgTimbre = extractAvgTimbre(soulAndReggae)

# classicPopAndRockVarTimbre = extractAvgTimbre(classicPopAndRock)
# punkVarTimbre = extractVarTimbre(punk)
# folkVarTimbre = extractVarTimbre(folk)
# popVarTimbre = extractVarTimbre(classicPopAndRock)
# danceAndElectronicaVarTimbre = extractVarTimbre(danceAndElectronica)
# metalVarTimbre = extractVarTimbre(metal)
# jazzVarTimbre = extractVarTimbre(jazz)
# classicalVarTimbre = extractVarTimbre(classical)
# hipHopVarTimbre = extractVarTimbre(hipHop)
# soulAndReggaeVarTimbre = extractVarTimbre(soulAndReggae)

# fig = plt.figure(figsize=(12, 9))
# ax = Axes3D(fig, elev=21, azim=-136)
# ax.scatter(dataframe['Top_Tail'], dataframe['Middle_Segment'], dataframe['Bottom_Tail'], 
#            c=labels.astype(np.float))
# ax.set_xlabel('TopTail')
# ax.set_ylabel('MiddleSegment')
# ax.set_zlabel('BottomTail')
# plt.show()

# fig = px.scatter(avgTimbreSample, x=avgTimbre1, y=avgTimbre2, color='genre')
# fig.show()

# fig = px.box(avgTimbreSample, x="genre", y=avgTimbre1)
# fig.show()